In [1]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ximenml0613/raw/master/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ximenml0613/raw/master/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7ffa5ec9e048>)

In [2]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [4]:
s = train_df.isna().sum()
# filter: 帶入跟資料大小依樣的true/false, true:留 false:滾
s[s > 0].sort_values(ascending=False)

Cabin       687
Age         177
Embarked      2
dtype: int64

In [5]:
s = test_df.isna().sum()
# filter: 帶入跟資料大小依樣的true/false, true:留 false:滾
s[s > 0].sort_values(ascending=False)

Cabin    327
Age       86
Fare       1
dtype: int64

In [6]:
# 數值類型: 中位數
med = train_df.median()
med = med.drop(["Pclass"])
train_df = train_df.fillna(med)
test_df = test_df.fillna(med)

In [7]:
# 類別類型: 最常出現
counts = train_df["Embarked"].value_counts()
most = counts.idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)

In [8]:
n = "Braund, Mr. Owen Harris"
n = n.split(",")[-1].split(".")[0]
n.strip()

'Mr'

In [9]:
import re
n = "Braund, Mr. Owen Harris"
# \w: A~Z a-z \s: blank +:>=1 *:>=0
# \w+: Braund: 一個以上a-z
# \s*: 零個以上的空白
# (\w+): Mr, ()是指這是我的target
# \.: .是有特殊用途的, 所以我\.: 就是.
# [\s\w]+:  Owen Harris \s or \w 出現一次以上
# group: 找出()
pat = r"\w+,\s*(\w+)\.[\s\w]+"
re.search(pat, n).group(1)

'Mr'

In [10]:
def namecut(n):
    reserved = ["Mr", "Miss", "Mrs", "Master"]
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in reserved:
        return n
train_df["Name"] = train_df["Name"].apply(namecut)
train_df["Name"]

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886    None
887    Miss
888    Miss
889      Mr
890      Mr
Name: Name, Length: 891, dtype: object

In [11]:
test_df["Name"] = test_df["Name"].apply(namecut)

In [12]:
x_train = train_df.drop(["PassengerId", "Survived", "Ticket", "Cabin"],
                        axis=1)
y_train = train_df["Survived"]
x_test = test_df.drop(["PassengerId", "Ticket", "Cabin"],
                      axis=1)

In [13]:
x_train

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,Mr,male,22.0,1,0,7.2500,S
1,1,Mrs,female,38.0,1,0,71.2833,C
2,3,Miss,female,26.0,0,0,7.9250,S
3,1,Mrs,female,35.0,1,0,53.1000,S
4,3,Mr,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,2,None,male,27.0,0,0,13.0000,S
887,1,Miss,female,19.0,0,0,30.0000,S
888,3,Miss,female,28.0,1,2,23.4500,S
889,1,Mr,male,26.0,0,0,30.0000,C


In [14]:
# Step2. One-Hot
x_train = pd.get_dummies(x_train)
x_train

,Pclass,Age,SibSp,Parch,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,0,0,0,0,1,0,0,1
887,1,19.0,0,0,30.0000,0,1,0,0,1,0,0,0,1
888,3,28.0,1,2,23.4500,0,1,0,0,1,0,0,0,1
889,1,26.0,0,0,30.0000,0,0,1,0,0,1,1,0,0


In [15]:
x_test = pd.get_dummies(x_test)
x_test

,Pclass,Age,SibSp,Parch,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,0,1,0,0,1,0,1,0
1,3,47.0,1,0,7.0000,0,0,0,1,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,0,1,0,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,0,1,0,0,1,0,0,1
4,3,22.0,1,1,12.2875,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,28.0,0,0,8.0500,0,0,1,0,0,1,0,0,1
414,1,39.0,0,0,108.9000,0,0,0,0,1,0,1,0,0
415,3,38.5,0,0,7.2500,0,0,1,0,0,1,0,0,1
416,3,28.0,0,0,8.0500,0,0,1,0,0,1,0,0,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier()
params = {
    "n_estimators":range(50, 70),
    "max_depth":range(6, 10)
}
grid = GridSearchCV(clf, params, cv=10, n_jobs=4)
grid.fit(x_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

In [17]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=65, max_depth=8)
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)
print("10 scores:", scores)
print("average:", np.average(scores))

10 scores: [0.77777778 0.85393258 0.75280899 0.8988764  0.88764045 0.83146067
 0.84269663 0.7752809  0.87640449 0.83146067]
average: 0.8328339575530587


In [18]:
clf = RandomForestClassifier(n_estimators=65, max_depth=8)
clf.fit(x_train, y_train)
pre = clf.predict(x_test)
result = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
result.to_csv("rf.csv", encoding="utf-8", index=False)
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [19]:
pd.DataFrame({
    "Col":x_train.columns,
    "Imp":clf.feature_importances_
})

,Col,Imp
0,Pclass,0.103243
1,Age,0.129333
2,SibSp,0.065763
3,Parch,0.037956
4,Fare,0.164158
5,Name_Master,0.013854
6,Name_Miss,0.033126
7,Name_Mr,0.146984
8,Name_Mrs,0.031923
9,Sex_female,0.117106


In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)
pd.DataFrame(x_train_scale)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.271174,0.125,0.000000,0.014151,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.472229,0.125,0.000000,0.139136,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.321438,0.000,0.000000,0.015469,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.434531,0.125,0.000000,0.103644,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.434531,0.000,0.000000,0.015713,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.5,0.334004,0.000,0.000000,0.025374,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
887,0.0,0.233476,0.000,0.000000,0.058556,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
888,1.0,0.346569,0.125,0.333333,0.045771,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
889,0.0,0.321438,0.000,0.000000,0.058556,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [22]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
params = {
    "n_neighbors":range(5, 100),
}
grid = GridSearchCV(clf, params, cv=10, n_jobs=4)
grid.fit(x_train_scale, y_train)
print(grid.best_params_)
print(grid.best_score_)

{'n_neighbors': 22}
0.8193508114856428


In [23]:
clf = grid.best_estimator_
clf.fit(x_train_scale, y_train)
pre = clf.predict(x_test_scale)
result = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
result.to_csv("knn.csv", encoding="utf-8", index=False)
result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
